The load modeling tool produces a piece-wise linear load model of power as function of temperature.  The load model tool reads a CSV file containing timestamped samples of power and temperature data.  The data is separated into three groups, heating, cooling, and neither.  This latter category is called the base load, and corresponds to loads for temperature conditions between the heating and cooling balance temperatures, as shown in Figure 1.

![example.png](example.png "Example")
Figure 1: Example load model

In [1]:
import os, sys
currentdir = os.path.abspath('')
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

import pandas as pd
import src.weather.sensitivity as lm
data = lm.load_data(ifile=f'{parentdir}/data/weather/testdata.csv',datetime_col=0,power_col=1,temperature_col=2)
print(data[0:10])

                       power  temperature
datetime                                 
2014-10-01 00:00:00  1798.85        52.35
2014-10-01 00:05:00  1782.63        52.26
2014-10-01 00:10:00  1766.58        52.14
2014-10-01 00:15:00  1768.91        51.95
2014-10-01 00:20:00  1756.20        51.99
2014-10-01 00:25:00  1744.86        51.98
2014-10-01 00:30:00  1734.70        51.86
2014-10-01 00:35:00  1750.62        51.96
2014-10-01 00:40:00  1730.56        52.05
2014-10-01 00:45:00  1724.83        51.86


In [2]:
model = lm.get_model(data,datetime_col=0,power_col=1,temperature_col=2)
pd.options.display.float_format = '{:,.1f}'.format
print(model)

/Users/derins/opt/anaconda3/lib/python3.7/site-packages/pwlf/pwlf.py:1109: RuntimeWarning: invalid value encountered in double_scalars
  (self.fit_breaks[i+1]-self.fit_breaks[i])


     Pbase   Pheat   Pcool  Tmin  Theat  Tcool  Tmax  Sbase  Sheat  Scool
0  1,776.2 2,696.8 2,712.6  22.0   52.9   62.1  83.1    7.8  -31.0   43.0
1  1,692.0 2,644.0 2,496.0  21.6   52.2   60.8  81.9    5.7  -32.0   36.9
2  1,644.7 2,629.1 2,323.8  21.5   51.1   59.1  80.7    0.7  -33.3   31.3
3  1,642.0 2,667.7 2,192.7  21.3   49.4   57.4  78.8   -8.1  -35.4   27.1
4  1,684.3 2,775.2 2,149.0  20.9   49.2   56.6  77.5  -11.2  -37.0   24.2
5  1,789.9 2,974.7 2,192.6  20.5   52.1   57.8  75.7   -0.3  -37.4   22.5
6  1,989.0 3,274.7 2,296.0  20.4   53.3   58.7  75.7   -4.4  -38.8   18.7
7  2,156.4 3,492.5 2,524.9  20.2   53.8   58.8  81.2  -12.3  -38.8   17.8
8  2,241.8 3,560.2 2,717.4  20.2   54.6   59.9  83.8   -8.7  -37.7   20.9
9  2,305.4 3,568.5 2,919.8  21.6   55.0   61.9  86.6   -5.2  -37.3   25.6
10 2,341.3 3,523.2 3,151.1  23.2   55.5   63.3  91.1   -5.5  -35.9   30.0
11 2,354.2 3,448.5 3,358.9  24.6   55.6   65.1  95.8   -5.5  -34.5   33.5
12 2,367.4 3,443.4 3,492.9  24.8   53.

# Example results

In [3]:
import matplotlib.pyplot as plt
import numpy as np

This example computes the load for a single hour/temperature point.

In [4]:
h = 15
t = 95.0
p = lm.get_load(model,h,t)

KeyError: 15

This example computes the load for each hour of the day at a given temperature.

In [ ]:
h = range(24)
t = 95.0
p = lm.get_load(model,h,t)

plt.plot(h,p)
plt.xlabel("Hour of day")
plt.ylabel("Load (MW)")
plt.title("Example 2: Load at 95 degF")
plt.grid()
plt.show()

This example computes the load profile for an hour of the day.

In [ ]:
h = 15
t = np.arange(25,95)
p = lm.get_load(model,h,t)

plt.plot(t,p)
plt.xlabel("Temperature (degF)")
plt.ylabel("Load (MW)")
plt.title("Example 3: Load at 15:00h")
plt.grid()
plt.show()

This example computes the normalized load shape for various temperature conditions.

In [ ]:
Tmin = model["Tmin"].min()
Tmax = model["Tmax"].max()
Tavg = model["Theat"].mean()/2 + model["Tcool"].mean()/2
h = range(24)
t = {f"Winter ({Tmin:.0f}degF)":Tmin,
     f"Spring ({Tavg:.0f}degF)":Tavg,
     f"Summer ({Tmax:.0f}degF)":Tmax}
p = lm.get_load(model,h,t.values(),normalize=True)

plt.plot(p)
plt.xlabel("Hour of day")
plt.ylabel("Load (MW)")
plt.title("Example 4: Load shapes")
plt.grid()
plt.legend(t.keys())
plt.show()

This example outputs the normalized load profiles for different times of day.

In [ ]:
h = [6,12,18]
t = range(25,95)
p = lm.get_load(model,h,t,convert=np.array,normalize=True).transpose()

plt.plot(p)
plt.xlabel("Temperature (degF)")
plt.ylabel("Load (MW)")
plt.title("Example 5: Load profiles")
plt.legend(list(map(lambda x:f"{x}:00",h)))
plt.grid()
plt.show()